### Data Model

`SeasonData`

* `season_name` (string)
* `num_rounds` (int)
* `players` (list[PlayerData])

`PlayerData`

* `player` (Player)
* `prices` (list[float])
* `scores` (list[float])
* `status` (list[PlayerStatus])

`Player`

* `name` (string)
* `position` (Position)

`Position` (enum)

* `GOALKEEPER`
* `WINGER`
* `DEFENDER`
* `MIDFIELDER`
* `FORWARD`
* `COACH`

`PlayerStatus` (enum)

* `POSSIBLE`
* `INJURED`
* `NULL`
* `DOUBT`
* `SUSPENDED`
* `UNKNOWN`

In [1]:
from enum import Enum

class Position(Enum):
    
    GOALKEEPER = 'Goleiro'
    WINGER = 'Lateral'
    DEFENDER = 'Zagueiro'
    MIDFIELDER = 'Meia'
    FORWARD = 'Atacante'
    COACH = 'Técnico'
    
    def __str__(self):
        return self.value

assert '{}'.format(Position.GOALKEEPER) == 'Goleiro'
assert Position.GOALKEEPER == Position('Goleiro')
assert len(Position) == 6

In [2]:
from collections import namedtuple

class Player(namedtuple('Player', ['name', 'position'])):
    __slots__ = ()
    
    def __str__(self):
        return '{} ({})'.format(*self)

assert Player('x', Position.GOALKEEPER) == Player('x', Position.GOALKEEPER)
assert Player('x', Position.GOALKEEPER) != Player('y', Position.GOALKEEPER)
assert str(Player('x', Position.GOALKEEPER)) == 'x (Goleiro)'

In [3]:
class PlayerStatus(Enum):
    
    POSSIBLE = 'Provável'
    INJURED = 'Contundido'
    NULL = 'Nulo'
    DOUBT = 'Dúvida'
    SUSPENDED = 'Suspenso'
    UNKNOWN = 'Desconhecido'
    
    def __str__(self):
        return self.value

assert '{}'.format(PlayerStatus.POSSIBLE) == 'Provável'
assert PlayerStatus.POSSIBLE == PlayerStatus('Provável')
assert len(PlayerStatus) == 6

In [4]:
class PlayerData:
    
    def __init__(self, player, prices, scores, status):
        self.player = player
        self.prices = prices
        self.scores = scores
        self.status = status
    
    def __str__(self):
        return str(self.player)

class SeasonData:
    
    def __init__(self, season_name, num_rounds, players):
        self.season_name = season_name
        self.num_rounds = num_rounds
        self.players = players
    
    def __str__(self):
        return '{} (rounds {}, players {})'.format(self.season_name,
                                                   self.num_rounds,
                                                   len(self.players))

In [5]:
import csv
import json
import os

CARTOLA_HOME = os.path.abspath('..')

print(CARTOLA_HOME)

if not os.path.isdir(CARTOLA_HOME):
    raise Exception('Cartola project is missing!')

CARTOLA_DATA = os.path.join(CARTOLA_HOME, 'data')

print(CARTOLA_DATA)

if not os.path.isdir(CARTOLA_DATA):
    raise Exception('Cartola data is missing!')

/home/cavani/Workspace/cartola-game
/home/cavani/Workspace/cartola-game/data


In [6]:
game_file = os.path.join(CARTOLA_DATA, '2015', 'game.json')

if not os.path.isfile(game_file):
    raise Exception('file not found: ' + game_file)

In [7]:
def load_game(game_file):
    with open(game_file) as f:
        g = json.load(f)
    return g['game_name'], g['game_season'], g['num_rounds']

game_title, season_name, num_rounds = load_game(game_file)

game_title, season_name, num_rounds

('Brasileirão 2015', '2015', 38)

In [8]:
players_file = os.path.join(CARTOLA_DATA, '2015', 'players.csv')

if not os.path.isfile(players_file):
    raise Exception('file not found: ' + players_file)

In [9]:
def load_players(players_file):
    players = dict()
    with open(players_file, newline='') as f:
        reader = csv.reader(f)
        next(reader) # spkip header
        for row in reader:
            id, name, position_name = row
            try:
                position = Position(position_name)
            except ValueError:
                print('[{}] Invalid position: {}'.format(i, position_name))
                continue
            players[id] = Player(name, position)
    return players

players = load_players(players_file)

len(players)

1041

In [10]:
list(players.items())[:10]

[('759',
  Player(name='Pablo Dyego da Silva Rosa', position=<Position.FORWARD: 'Atacante'>)),
 ('822',
  Player(name='Yago Felipe da Costa Rocha', position=<Position.MIDFIELDER: 'Meia'>)),
 ('550',
  Player(name='Luiz Eduardo dos Santos Gonzaga', position=<Position.FORWARD: 'Atacante'>)),
 ('800',
  Player(name='Donato Antônio Silva Neto', position=<Position.DEFENDER: 'Zagueiro'>)),
 ('375',
  Player(name='Alejandro Martinuccio', position=<Position.FORWARD: 'Atacante'>)),
 ('1008',
  Player(name='Kaio Silva Mendes', position=<Position.MIDFIELDER: 'Meia'>)),
 ('792',
  Player(name='Matheus da Silva Fortes', position=<Position.FORWARD: 'Atacante'>)),
 ('557',
  Player(name='Alex Roberto Santana Rafael ', position=<Position.GOALKEEPER: 'Goleiro'>)),
 ('738', Player(name='Thalles Lima', position=<Position.FORWARD: 'Atacante'>)),
 ('895',
  Player(name='Jaílson Marques Siqueira', position=<Position.MIDFIELDER: 'Meia'>))]

In [11]:
season_file = os.path.join(CARTOLA_DATA, '2015', 'season.csv')

if not os.path.isfile(season_file):
    raise Exception('file not found: ' + season_file)

In [12]:
def load_data(players, season_file, num_rounds):
    players_data = dict()
    
    with open(season_file, newline='') as f:
        reader = csv.reader(f)
        next(reader) # skip header
        for i, row in enumerate(reader):
            player_id = row[0]
            round_i = row[1]
            status_name = row[2]
            price = row[3]
            score = row[4]
            
            if player_id not in players:
                print('[{}] Player not found: {}'.format(i, player_id))
                continue
            try:
                round_i = int(round_i) - 1
            except ValueError:
                print('[{}] Invalid round: {}'.format(i, round_i))
                continue
            if round_i < 0 or round_i >= num_rounds:
                print('[{}] Round out of bounds: {} ({})'.format(i, round_i + 1, num_rounds))
                continue
            try:
                status = PlayerStatus(status_name)
            except ValueError:
                print('[{}] Invalid status: {}'.format(i, status_name))
                continue
            try:
                price = float(price)
            except ValueError:
                print('[{}] Invalid price: {}'.format(i, price))
                continue
            try:
                score = float(score)
            except ValueError:
                print('[{}] Invalid score: {}'.format(i, score))
                continue
            
            if player_id not in players_data:
                _player = players[player_id]
                _prices = [0.0] * num_rounds
                _scores = [0.0] * num_rounds
                _status = [PlayerStatus.UNKNOWN] * num_rounds
                players_data[player_id] = PlayerData(_player,
                                                     _prices,
                                                     _scores,
                                                     _status)
                
            player_data = players_data[player_id]
            player_data.prices[round_i] = price
            player_data.scores[round_i] = score
            player_data.status[round_i] = status
    
    return list(players_data.values())

players_data = load_data(players, season_file, num_rounds)

len(players_data)

1041

In [13]:
for p in players_data[:10]:
    print(p)

Pablo Dyego da Silva Rosa (Atacante)
Yago Moreira Silva (Atacante)
Geuvânio Santos Silva (Atacante)
Patrick Bezerra do Nascimento (Meia)
Samuel Portugal (Goleiro)
Luiz Felipe Ventura dos Santos (Goleiro)
Matheus da Silva Fortes (Atacante)
Alex Roberto Santana Rafael  (Goleiro)
Charles Fernando Basílio da Silva (Meia)
Petros Mateus dos Santos Araújo (Meia)


In [14]:
def load_season(season_name, data_path=CARTOLA_DATA):
    season_path = os.path.join(data_path, season_name)
    game_file = os.path.join(season_path, 'game.json')
    players_file = os.path.join(season_path, 'players.csv')
    season_file = os.path.join(season_path, 'season.csv')
    
    if not os.path.isfile(game_file):
        raise Exception('file not found: ' + game_file)
    if not os.path.isfile(players_file):
        raise Exception('file not found: ' + players_file)
    if not os.path.isfile(season_file):
        raise Exception('file not found: ' + season_file)
    
    game_title, season_name, num_rounds = load_game(game_file)
    players = load_players(players_file)
    players_data = load_data(players, season_file, num_rounds)
    
    return SeasonData(season_name, num_rounds, players_data)

season = load_season('2015')

print(season)

2015 (rounds 38, players 1041)


### Game Model

`Team`

* `formation` (Formation)
* `players` (Player)

`Formation` (enum)

* properties:
    * `label` (string)
    * `positions` (list[PositionSlot])
* members:
    * `F343`
    * `F352`
    * `F433`
    * `F442`
    * `F451`
    * `F532`
    * `F541`

`PositionSlot`:

* `position` (Position)
* `size` (int)


In [15]:
from collections import namedtuple

PositionSlot = namedtuple('PositionSlot', ['position', 'size'])

p = PositionSlot(Position.GOALKEEPER, 1)

print(p)

PositionSlot(position=<Position.GOALKEEPER: 'Goleiro'>, size=1)


In [16]:
from enum import Enum

GOALKEEPER = Position.GOALKEEPER
WINGER = Position.WINGER
DEFENDER = Position.DEFENDER
MIDFIELDER = Position.MIDFIELDER
FORWARD = Position.FORWARD
COACH = Position.COACH

def f(text, *p): return text, list(map(PositionSlot._make, p))

class Formation(Enum):
    
    F343 = f('3-4-3',
             (GOALKEEPER, 1),
             (DEFENDER, 3),
             (MIDFIELDER, 4),
             (FORWARD, 3),
             (COACH, 1))
    
    F352 = f('3-5-2',
             (GOALKEEPER, 1),
             (DEFENDER, 3),
             (MIDFIELDER, 5),
             (FORWARD, 2),
             (COACH, 1))

    F433 = f('4-3-3',
             (GOALKEEPER, 1),
             (WINGER, 2),
             (DEFENDER, 2),
             (MIDFIELDER, 3),
             (FORWARD, 3),
             (COACH, 1))
    
    F442 = f('4-4-2',
             (GOALKEEPER, 1),
             (WINGER, 2),
             (DEFENDER, 2),
             (MIDFIELDER, 4),
             (FORWARD, 2),
             (COACH, 1))
    
    F451 = f('4-5-1',
             (GOALKEEPER, 1),
             (WINGER, 2),
             (DEFENDER, 2),
             (MIDFIELDER, 5),
             (FORWARD, 1),
             (COACH, 1))

    F532 = f('5-3-2',
             (GOALKEEPER, 1),
             (WINGER, 2),
             (DEFENDER, 3),
             (MIDFIELDER, 3),
             (FORWARD, 2),
             (COACH, 1))
    
    F541 = f('5-4-1',
             (GOALKEEPER, 1),
             (WINGER, 2),
             (DEFENDER, 3),
             (MIDFIELDER, 3),
             (FORWARD, 2),
             (COACH, 1))
    
    @property
    def label(self):
        return self.value[0]

    @property
    def positions(self):
        return self.value[1]
    
    def __str__(self):
        positions = [str(p.position) for p in self.positions for _ in range(p.size)]
        return '{}\n\n{}'.format(self.label, '\n'.join(positions))

In [17]:
len(Formation)

7

In [18]:
Formation.F343

<Formation.F343: ('3-4-3', [PositionSlot(position=<Position.GOALKEEPER: 'Goleiro'>, size=1), PositionSlot(position=<Position.DEFENDER: 'Zagueiro'>, size=3), PositionSlot(position=<Position.MIDFIELDER: 'Meia'>, size=4), PositionSlot(position=<Position.FORWARD: 'Atacante'>, size=3), PositionSlot(position=<Position.COACH: 'Técnico'>, size=1)])>

In [19]:
print(Formation.F343)

3-4-3

Goleiro
Zagueiro
Zagueiro
Zagueiro
Meia
Meia
Meia
Meia
Atacante
Atacante
Atacante
Técnico


In [20]:
F343 = Formation.F343
F352 = Formation.F352
F433 = Formation.F433
F442 = Formation.F442
F451 = Formation.F451
F532 = Formation.F532
F541 = Formation.F541

In [21]:
from collections import defaultdict

def team_is_valid(formation, players, verbose=False):
    if not isinstance(players, set):
        players = set(players) # unique
    count = defaultdict(int)
    for player in players:
        count[player.position] += 1
    valid = True
    for position, size in formation.positions:
        if size != count[position]:
            valid = False
            if not verbose:
                break
            print('Missing {} (expected {}): {}' \
                  .format(position, size, count[position]))
    return valid

print('no players\n')
assert not team_is_valid(F343, [], True)

print('\nrepeated players\n')

invalid_team = [Player('g', GOALKEEPER)]
invalid_team += [Player('d', DEFENDER)] * 3
invalid_team += [Player('m', MIDFIELDER)] * 4
invalid_team += [Player('f', FORWARD)] * 3
invalid_team += [Player('c', COACH)]

assert not team_is_valid(F343, invalid_team, True)

print('\ngood players\n\n(empty)')

valid_team = [Player('a', GOALKEEPER)]
valid_team += [Player('d {}'.format(i+1), DEFENDER) for i in range(3)]
valid_team += [Player('m {}'.format(i+1), MIDFIELDER) for i in range(4)]
valid_team += [Player('f {}'.format(i+1), FORWARD) for i in range(3)]
valid_team += [Player('e', COACH)]

assert team_is_valid(F343, valid_team, True)

no players

Missing Goleiro (expected 1): 0
Missing Zagueiro (expected 3): 0
Missing Meia (expected 4): 0
Missing Atacante (expected 3): 0
Missing Técnico (expected 1): 0

repeated players

Missing Zagueiro (expected 3): 1
Missing Meia (expected 4): 1
Missing Atacante (expected 3): 1

good players

(empty)


In [22]:
class Team:
    
    def __init__(self, formation, players, validate=True):
        if validate and not team_is_valid(formation, players):
            raise Exception('Team is invalid!')
        self.formation = formation
        self.players = set(players)
    
    def __str__(self):
        formation = self.formation.label
        players = defaultdict(list)
        for player in self.players:
            players[player.position].append(player.name)
        team = ('{} -> {}'.format(pos, name)
                for pos, _ in self.formation.positions
                for name in sorted(players[pos]))
        return '{}\n\n{}'.format(formation, '\n'.join(team))

    
t = Team(F343, valid_team)
print(t)

3-4-3

Goleiro -> a
Zagueiro -> d 1
Zagueiro -> d 2
Zagueiro -> d 3
Meia -> m 1
Meia -> m 2
Meia -> m 3
Meia -> m 4
Atacante -> f 1
Atacante -> f 2
Atacante -> f 3
Técnico -> e
